In [1]:
import os
import numpy as np
import tensorflow as tf

In [3]:
# Load the TFLite model and allocate tensors.
# Let's load the TFLite model we saved to "./saved_tflite_models/model_2.tflite" above.
model_complexity=0
interpreter = tf.lite.Interpreter(model_path="model_2.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [4]:
!pip install opendatasets

In [5]:
import opendatasets as od
import pandas as pd

od.download('https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign')

100%|██████████| 612M/612M [00:04<00:00, 146MB/s]


In [6]:
batch_size = 32
img_height = 224
img_width = 224
test = tf.keras.preprocessing.image_dataset_from_directory(
  "gtsrb-german-traffic-sign/Test",
  labels=None,
  seed=123,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=1)

Found 12630 files belonging to 1 classes.


In [7]:
import numpy as np
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
test = test.map(lambda x: (normalization_layer(x)))

In [9]:
import time
start_time = time.time()
output_data=[]
for input_data in test:
  input_data = np.float32(input_data)
  interpreter.set_tensor(input_details[0]['index'], input_data)

  # "invoke()" sends the data through the model.
  interpreter.invoke()
  output_data += [interpreter.get_tensor(output_details[0]['index'])]
  #predicted_label_batch = class_names[output_data]
time_prediction = time.time() - start_time
print(time_prediction)

201.92879366874695


the time to make the prediction is way worst 80>201

In [10]:
output_data=np.argmax(output_data,axis=-1)

In [11]:
results=pd.read_csv("gtsrb-german-traffic-sign/Test.csv",sep=",")

In [12]:
acc=0
for n in range(len(output_data)):
  if int(output_data[n][0])==int(results["ClassId"][n]) and int(results["ClassId"][n])<6:
    acc+=1
print(acc,len(results[results["ClassId"].astype(int)<6]["ClassId"]), acc*100/len(results[results["ClassId"]<10]["ClassId"]))

339 3270 7.0625


we have better accuracy on this model

In [13]:
print("model_size_compressed =",os.path.getsize('model_2.tflite'))


model_size_compressed = 8916092


8.9MB of size, which is evier than the orther one